# Python Descriptor 完整指南
## 1. 基礎概念和核心原理

### 1.1 什麼是 Descriptor？
- 定義和基本概念
- Descriptor 的優勢和使用時機
- Descriptor vs 普通屬性的差異

### 1.2 Descriptor 協議方法
Descriptor 是一個實現了以下特定方法的類：
- __get__(self, obj, owner=None)：當訪問屬性時被調用
- __set__(self, obj, value)：當設置屬性時被調用
- __delete__(self, obj)：當刪除屬性時被調用
- __set_name__(self, owner, name)：當描述符被分配給類屬性時被調用（Python 3.6+）

### 1.3 描述符的類型
- 數據描述符（data descriptor）
- 非數據描述符（non-data descriptor）
- 類型差異和優先級

## 2. 工作原理
### 2.1 屬性查找順序
- 完整的查找流程
- 優先級規則
- 實例屬性 vs 描述符

### 2.2 記憶體管理
- 值的存儲方式
- WeakKeyDictionary 的使用
- 避免記憶體洩漏

## 3. 基礎應用場景
### 3.1 屬性訪問控制
- 只讀屬性（ReadOnly 例子）
- 私有屬性
- 受限制的屬性

### 3.2 類型檢查和驗證
- 基本類型檢查（TypeChecked 例子）
- 值範圍驗證
- 自定義驗證規則

### 3.3 訪問日誌和權限控制
- 基本日誌記錄
- 權限控制
- 結合日誌和權限的綜合應用

### 3.4 計算屬性
- 基本計算屬性（Circle.area 例子）
- 緩存計算結果
- 動態計算屬性

## 4. 進階應用
### 4.1 ORM 實現
- 數據庫字段映射
- 類型轉換
- 驗證規則

### 4.2 惰性加載
- 延遲初始化
- 資源管理
- 性能優化

### 4.3 屬性代理
- 屬性轉發
- 組合模式
- 介面簡化

## 5. 最佳實踐
### 5.1 使用準則
- 何時使用 Descriptor
- 性能考慮
- 代碼組織

### 5.2 常見錯誤
- 實例屬性誤用
- 值存儲問題
- 記憶體管理問題

### 5.3 設計模式
- 單一職責原則
- 代碼重用
- 擴展性考慮

## 6. 實際案例分析
### 6.1 Django ORM
- 模型字段實現
- 驗證機制
- 關係處理

### 6.2 屬性管理系統
- 配置管理
- 權限控制
- 審計日誌

### 6.3 緩存系統
- 計算緩存
- 資源管理
- 性能優化

## 7. 總結與參考
### 7.1 使用建議
- 最佳實踐總結
- 常見陷阱
- 性能考慮

### 7.2 進階閱讀
- 相關資源
- 官方文檔
- 推薦閱讀

## 1. 基礎概念和核心原理
### 1.1 什麼是 Descriptor？
#### 1.1.1 Descriptor定義和基本概念
- Descriptor 是 Python 中一種強大的屬性管理工具，允許我們自定義當訪問類屬性時的行為(用於控制屬性的讀取與寫入行為)。
- 它的核心在於能在屬性存取時插入邏輯，比如驗證數值或記錄操作。
- 簡單來說，任何實現了(以下四個特殊方法) descriptor 協議方法的對象都是 descriptor。

#### 1.1.2 Descriptor 的優勢和使用時機
1. 實時執行：每次訪問屬性時都會執行特定邏輯
2. 狀態感知：可以訪問和使用實例的狀態
3. 代碼重用：相同的邏輯可以在多個類中使用
4. 更好的封裝：將屬性相關的邏輯完整地封裝在一個地方
5. 提高維護性：修改邏輯時只需要修改一個地方

#### 1.1.3 Descriptor vs 普通屬性的差異


In [ ]:
class MyClass:
    print("有人正在讀取這個值")
    x = 42

# 測試
obj_1 = MyClass()
obj_2 = MyClass()
print(obj_1.x)  # 輸出: 42
print(obj_2.x)

In [ ]:
# 一般類屬性
class MyClass_1:
    print("這個只在類定義時執行一次")
    x = 42

# 使用 descriptor
class LoggedValue:
    def __get__(self, obj, owner=None):
        print("這個在每次訪問時都會執行")
        return 42

class MyClass_2:
    x = LoggedValue()

# 測試
obj_1 = MyClass_1()
obj_2 = MyClass_2()


print("==================================")
print(obj_1.x)  # 會打印日誌
print("----------------------------------")
print(obj_1.x)  # 再次打印日誌
print("==================================")
print(obj_2.x)  # 會打印日誌
print("----------------------------------")
print(obj_2.x)  # 再次打印日誌

### 1.2 Descriptor 協議方法
- __get__(self, obj, owner=None)：當讀取屬性時調用
- __set__(self, obj, value)：當設置屬性時調用
- __delete__(self, obj)：當刪除屬性時調用
- __set_name__(self, owner, name)：當描述符被分配給類屬性時調用

In [5]:
class Descriptor:
    def __get__(self, obj, owner=None):
        """
        當訪問屬性時調用
        obj: 擁有該屬性的實例
        owner: 擁有該屬性的類
        """
        print(f"正在獲取值")
        print(f"- 實例是：{obj}")
        print(f"- 類是：{owner}")
        return self._value

    def __set__(self, obj, value):
        """
        當設置屬性值時調用
        obj: 擁有該屬性的實例
        value: 要設置的值
        """
        print(f"正在設置值：{value}")
        self._value = value

    def __delete__(self, obj):
        """
        當刪除屬性時調用
        obj: 擁有該屬性的實例
        """
        print("正在刪除值")
        del self._value

    def __set_name__(self, owner, name):
        """
        當 descriptor 被指派給類屬性時調用（Python 3.6+）
        owner: 擁有該屬性的類
        name: 屬性名稱
        """
        print(f"設置名稱：{name}")
        self._name = name

#### 1.2.1 __get__(self, obj, owner=None) 方法
get 方法只是管理訪問，只是監控屬性的訪問
有三個功能：
1. 最簡單的只讀 descriptor：
2. 計算屬性的 descriptor
3. 只需要管理訪問的 descriptor

應用方面：
純訪問管理，適用於日誌、權限控制等場景

主要參數解析：
- self: descriptor 實例本身
- obj: 訪問這個 descriptor 的實例
- owner: 這個 descriptor 所屬的類



1. 最簡單的只讀 descriptor：

就像一個類如果實現了 __len__ 方法就可以使用 len() 函數一樣，實現了 descriptor 協議方法的類就可以控制其作為類屬性時的行為。這就是 "協議" 的概念 - 遵循特定的規則來獲得特定的功能。

In [ ]:
# 這是一個 descriptor，因為它實現了 __get__ 方法
class SimpleDescriptor:
    def __get__(self, obj, owner=None):
        return 42

# 這不是一個 descriptor，因為它沒有實現任何 descriptor 協議方法
class NotDescriptor:
    def normal_method(self):
        return 42

# 使用示例
class MyClass:
    x = SimpleDescriptor()    # x 是一個 descriptor，它實現了 __get__ 方法
    y = NotDescriptor()       # y 是一個普通物件

obj = MyClass()
print(obj.x)    # 當訪問 obj.x 時，會調用 __get__ 方法，輸出 42
print(obj.y)    # 當訪問 obj.y 時，直接返回這個實例本身

42


2. 計算屬性的 descriptor：
get 方法 依賴其他實例屬性進行運算

- 可以把計算邏輯封裝在描述符中
- 使用起來像屬性一樣自然
- 每次訪問都重新計算，總是得到最新結果
- 正確處理了類訪問和實例訪問的不同情況
- 訪問實例屬性：適用於需要動態計算、數據轉換等場景

In [ ]:
class Circle:                             # 創建圓並設置半徑。
    def __init__(self, radius):
        self.radius = radius

    class AreaDescriptor:
        def __get__(self, obj, owner=None):
            print(f"self: {self}")           # 描述符實例
            print(f"obj: {obj}")             # Circle 實例
            print(f"owner: {owner}")         # Circle 類

            if obj is None:
                return self
            return 3.14 * obj.radius ** 2

    area = AreaDescriptor()

# 測試
circle_1 = Circle(5)

print("\n通過實例訪問：")
print(circle_1.area)    # 實例訪問 我們要計算這個具體圓的面積

print("\n通過類訪問：")
print(Circle.area)    # 類訪問 我們可能想查看這個屬性本身的資訊


通過實例訪問：
self: <__main__.Circle.AreaDescriptor object at 0x000002AB8F1D2270>
obj: <__main__.Circle object at 0x000002AB8F1D23C0>
owner: <class '__main__.Circle'>
78.5

通過類訪問：
self: <__main__.Circle.AreaDescriptor object at 0x000002AB8F1D2270>
obj: None
owner: <class '__main__.Circle'>


3. 只需要管理訪問的 descriptor：

In [18]:
class LoggingDescriptor:
    def __get__(self, obj, owner=None):
        print(f"正在訪問屬性")
        return "hello"

class MyClass:
    x = LoggingDescriptor()

# 測試
obj = MyClass()
print(obj.x)  # 輸出: 正在訪問屬性, hello

正在訪問屬性
hello


In [ ]:
class ExampleDescriptor:
    def __init__(self):             # 添加初始化方法  避免在第一次訪問時出現屬性不存在的錯誤
        self._value = None          # 初始化時創建 self._value 屬性
        print("Descriptor 初始化")
        print("--------------------------------")
        print(f"初始值：{self._value}")
        print("--------------------------------")
        print()

    def __get__(self, obj, owner=None):
        print(f"1. 當前實例 obj: {obj}")
        print(f"2. 實例所屬類 owner: {owner}")
        print("--------------------------------")
        print(f"返回的值：{self._value}")
        print("--------------------------------")

        # 通過類訪問的情況
        if obj is None:
            print("3. 這是通過'類'訪問的情況")
            return self

        # 通過實例訪問的情況
        print("3. 這是通過實例訪問的情況")
        return self._value

    def __set__(self, obj, value):  # 添加設置值的方法  用於處理賦值操作  沒有這個方法就無法設置值
        print(f"設置值: {value}")
        self._value = value

class MyClass:
    x = ExampleDescriptor()

# 測試代碼
my_obj = MyClass()

print("\n通過實例訪問並設置值:")
my_obj.x = 42                   # 正確的賦值方式
print(f"獲取值: {my_obj.x}")

print("\n通過類訪問:")
print(MyClass.x)

Descriptor 初始化
--------------------------------
初始值：None
--------------------------------


通過實例訪問並設置值:
設置值: 42
1. 當前實例 obj: <__main__.MyClass object at 0x000002AB8F1D0EC0>
2. 實例所屬類 owner: <class '__main__.MyClass'>
--------------------------------
返回的值：42
--------------------------------
3. 這是通過實例訪問的情況
獲取值: 42

通過類訪問:
1. 當前實例 obj: None
2. 實例所屬類 owner: <class '__main__.MyClass'>
--------------------------------
返回的值：42
--------------------------------
3. 這是通過'類'訪問的情況


In [ ]:
# descriptor 是實現了特定方法的類
# 最常見的是 __get__、__set__、__delete__ 這三個方法

class SimpleDescriptor:
    def __get__(self, obj, owner=None):
        print(f"取值，物件是：{obj}，類是：{owner}")
        return 42

class MyClass:
    x = SimpleDescriptor()  # 將 descriptor 作為類屬性

# 使用
obj = MyClass()
print("--------------------------------")
print(obj.x)  # 會觸發 __get__ 方法
print("--------------------------------")

--------------------------------
取值，物件是：<__main__.MyClass object at 0x000002CF2300F1D0>，類是：<class '__main__.MyClass'>
42


#### 1.2.2 __set__(self, obj, value) 方法
主要參數解析：

- self: descriptor 實例本身
- obj: 要設置值的實例
- value: 要設置的新值

In [ ]:
class ExampleDescriptor:
    def __set__(self, obj, value):
        print(f"1. 當前實例 obj: {obj}")
        print(f"2. 要設置的值 value: {value}")

        # 實現基本類型驗證
        if not isinstance(value, (int, float)):
            raise TypeError("只能設置數字!")

        # 存儲值
        self._value = value

class MyClass:
    x = ExampleDescriptor()

# 測試代碼
my_obj = MyClass()
print("設置合法值:")
my_obj.x = 42

print("\n設置非法值:")
try:
    my_obj.x = "hello"
except TypeError as e:
    print(f"錯誤: {e}")

#### 1.2.3 __delete__(self, obj) 方法
主要參數解析：

- self: descriptor 實例本身
- obj: 要刪除值的實例

In [ ]:
class ExampleDescriptor:
    def __delete__(self, obj):
        print(f"1. 當前實例 obj: {obj}")

        # 清除存儲的值
        if hasattr(self, '_value'):
            print("2. 刪除存儲的值")
            del self._value
        else:
            print("2. 沒有值可刪除")

class MyClass:
    x = ExampleDescriptor()

# 測試代碼
my_obj = MyClass()
my_obj.x = 42
print("刪除值:")
del my_obj.x

#### 1.2.4 __set_name__(self, owner, name) 方法 (Python 3.6+)
主要參數解析：

self: descriptor 實例本身
owner: descriptor 所屬的類
name: descriptor 在類中被指派的名稱

In [ ]:
class ExampleDescriptor:
    def __set_name__(self, owner, name):
        print(f"1. descriptor 所屬的類 owner: {owner}")
        print(f"2. descriptor 在類中的名稱 name: {name}")

        # 保存屬性名稱供後續使用
        self._name = name

class MyClass:
    # 當 Python 創建 MyClass 時，會自動調用 __set_name__
    x = ExampleDescriptor()
    y = ExampleDescriptor()

### 1.4 完整的實際應用示例

這個完整示例展示了：

- 如何使用所有 descriptor 協議方法
- 如何為不同實例存儲不同的值
- 如何進行數據驗證
- 如何處理錯誤情況
- 如何提供有意義的錯誤消息
- 如何在應用中實際使用 descriptor

這些方法共同工作，使得 descriptor 成為 Python 中一個強大的屬性管理工具。你可以用它來實現屬性驗證、惰性計算、訪問控制等多種功能。

In [ ]:
class ValidatedAttribute:
    def __init__(self, min_value=None, max_value=None):
        self.min_value = min_value
        self.max_value = max_value
        self._name = None
        self._values = {}  # 存儲不同實例的值

    def __set_name__(self, owner, name):
        print(f"初始化: 在 {owner.__name__} 類中設置屬性名 {name}")
        self._name = name

    def __get__(self, obj, owner=None):
        if obj is None:
            return self

        print(f"獲取: {obj} 的 {self._name} 值")
        return self._values.get(obj, None)

    def __set__(self, obj, value):
        print(f"設置: 在 {obj} 上將 {self._name} 設為 {value}")

        # 類型檢查
        if not isinstance(value, (int, float)):
            raise TypeError(f"{self._name} 必須是數字")

        # 範圍檢查
        if self.min_value is not None and value < self.min_value:
            raise ValueError(f"{self._name} 不能小於 {self.min_value}")
        if self.max_value is not None and value > self.max_value:
            raise ValueError(f"{self._name} 不能大於 {self.max_value}")

        # 存儲值
        self._values[obj] = value

    def __delete__(self, obj):
        print(f"刪除: {obj} 的 {self._name} 值")
        self._values.pop(obj, None)

class Person:
    age = ValidatedAttribute(min_value=0, max_value=150)
    height = ValidatedAttribute(min_value=0, max_value=300)

# 測試代碼
def test_validated_attribute():
    print("\n創建實例:")
    p1 = Person()
    p2 = Person()

    print("\n設置有效值:")
    p1.age = 25
    p1.height = 180
    p2.age = 30

    print("\n讀取值:")
    print(f"p1.age = {p1.age}")
    print(f"p1.height = {p1.height}")
    print(f"p2.age = {p2.age}")

    print("\n嘗試設置無效值:")
    try:
        p1.age = -5
    except ValueError as e:
        print(f"錯誤: {e}")

    try:
        p1.height = "不是數字"
    except TypeError as e:
        print(f"錯誤: {e}")

    print("\n刪除值:")
    del p1.age
    print(f"刪除後的 p1.age = {p1.age}")

test_validated_attribute()

初始化: 在 Person 類中設置屬性名 age
初始化: 在 Person 類中設置屬性名 height

創建實例:

設置有效值:
設置: 在 <__main__.Person object at 0x000002CF23060150> 上將 age 設為 25
設置: 在 <__main__.Person object at 0x000002CF23060150> 上將 height 設為 180
設置: 在 <__main__.Person object at 0x000002CF2300EA90> 上將 age 設為 30

讀取值:
獲取: <__main__.Person object at 0x000002CF23060150> 的 age 值
p1.age = 25
獲取: <__main__.Person object at 0x000002CF23060150> 的 height 值
p1.height = 180
獲取: <__main__.Person object at 0x000002CF2300EA90> 的 age 值
p2.age = 30

嘗試設置無效值:
設置: 在 <__main__.Person object at 0x000002CF23060150> 上將 age 設為 -5
錯誤: age 不能小於 0
設置: 在 <__main__.Person object at 0x000002CF23060150> 上將 height 設為 不是數字
錯誤: height 必須是數字

刪除值:
刪除: <__main__.Person object at 0x000002CF23060150> 的 age 值
獲取: <__main__.Person object at 0x000002CF23060150> 的 age 值
刪除後的 p1.age = None


In [ ]:
# 1. 基本的只讀屬性
class ReadOnly:
    def __init__(self, value):
        self._value = value

    def __get__(self, obj, owner=None):
        return self._value

    def __set__(self, obj, value):
        raise AttributeError("此屬性是只讀的")

class Config:
    VERSION = ReadOnly("1.0.0")
    API_KEY = ReadOnly("secret_key")

# 2. 私有屬性（帶命名檢查）
class PrivateAttribute:
    def __init__(self):
        self._value = None
        self._name = None

    def __set_name__(self, owner, name):
        self._name = name

    def __get__(self, obj, owner=None):
        if obj is None:
            return self
        if not self._check_access(obj):
            raise PermissionError(f"無法訪問私有屬性 {self._name}")
        return self._value

    def __set__(self, obj, value):
        if not self._check_access(obj):
            raise PermissionError(f"無法修改私有屬性 {self._name}")
        self._value = value

    def _check_access(self, obj):
        # 檢查是否從類的內部方法訪問
        import inspect
        frame = inspect.currentframe().f_back.f_back
        if frame is None:
            return False
        return frame.f_code.co_name in obj.__class__.__dict__

class User:
    _password = PrivateAttribute()

    def __init__(self, password):
        self._password = password

    def check_password(self, pwd):
        return self._password == pwd

# 3. 帶訪問日誌的屬性
class LoggedAttribute:
    def __init__(self):
        self._value = None
        self._name = None

    def __set_name__(self, owner, name):
        self._name = name

    def __get__(self, obj, owner=None):
        if obj is None:
            return self
        print(f"正在讀取 {obj.__class__.__name__}.{self._name}")
        return self._value

    def __set__(self, obj, value):
        print(f"正在設置 {obj.__class__.__name__}.{self._name} = {value}")
        self._value = value

# 4. 帶驗證和權限檢查的屬性
class SecureAttribute:
    def __init__(self, permission_level=0):
        self._value = None
        self._name = None
        self._permission_level = permission_level

    def __set_name__(self, owner, name):
        self._name = name

    def __get__(self, obj, owner=None):
        if obj is None:
            return self
        if not hasattr(obj, '_user_level'):
            raise AttributeError("對象必須有 _user_level 屬性")
        if obj._user_level < self._permission_level:
            raise PermissionError("權限不足")
        return self._value

    def __set__(self, obj, value):
        if not hasattr(obj, '_user_level'):
            raise AttributeError("對象必須有 _user_level 屬性")
        if obj._user_level < self._permission_level:
            raise PermissionError("權限不足")
        self._value = value

# 5. 帶緩存的延遲訪問控制
class CachedProperty:
    def __init__(self, expensive_function):
        self.expensive_function = expensive_function
        self._cache = {}
        self._name = expensive_function.__name__

    def __get__(self, obj, owner=None):
        if obj is None:
            return self
        if obj not in self._cache:
            print(f"計算 {self._name} 的值...")
            self._cache[obj] = self.expensive_function(obj)
        return self._cache[obj]

# 使用示例
def test_access_control():
    print("\n1. 測試只讀屬性:")
    config = Config()
    print(f"版本: {config.VERSION}")
    try:
        config.VERSION = "2.0.0"
    except AttributeError as e:
        print(f"嘗試修改只讀屬性: {e}")

    print("\n2. 測試私有屬性:")
    user = User("secret123")
    print("從方法內部訪問密碼:", user.check_password("secret123"))
    try:
        print("直接訪問密碼:", user._password)
    except PermissionError as e:
        print(f"訪問受限: {e}")

    print("\n3. 測試帶日誌的屬性:")
    class Employee:
        name = LoggedAttribute()
        salary = LoggedAttribute()

    emp = Employee()
    emp.name = "Alice"
    print(f"員工姓名: {emp.name}")

    print("\n4. 測試安全屬性:")
    class Document:
        _user_level = 1
        content = SecureAttribute(permission_level=2)

    doc = Document()
    try:
        doc.content = "機密信息"
    except PermissionError as e:
        print(f"寫入受限: {e}")

    print("\n5. 測試緩存屬性:")
    class DataProcessor:
        def expensive_calculation(self):
            # 模擬耗時計算
            import time
            time.sleep(1)
            return 42

        result = CachedProperty(expensive_calculation)

    dp = DataProcessor()
    print("第一次訪問:", dp.result)
    print("第二次訪問:", dp.result)

if __name__ == "__main__":
    test_access_control()

## 2. Descriptor 的工作原理
### 2.1 屬性查找順序
Python 在查找屬性時遵循以下順序：

1. __getattribute__ 方法
2. 數據描述符（同時實現了 __get__ 和 __set__）
3. 實例字典（__dict__）
4. 非數據描述符（只實現了 __get__）
5. 類字典
6. 父類字典
7. __getattr__ 方法（如果定義了）

### 2.2 數據描述符 vs 非數據描述符

In [ ]:
# 演示屬性查找順序
class DataDescriptor:
    def __get__(self, obj, owner=None):
        return "數據描述符"
    def __set__(self, obj, value):
        pass

class NonDataDescriptor:
    def __get__(self, obj, owner=None):
        return "非數據描述符"

class Example:
    data_desc = DataDescriptor()
    non_data_desc = NonDataDescriptor()

    def __init__(self):
        self.data_desc = "實例屬性 (data_desc)"
        self.non_data_desc = "實例屬性 (non_data_desc)"

# 測試
def test_lookup_order():
    obj = Example()

    # 數據描述符優先級高於實例屬性
    print(obj.data_desc)  # 輸出: "數據描述符"

    # 實例屬性優先級高於非數據描述符
    print(obj.non_data_desc)  # 輸出: "實例屬性 (non_data_desc)"

In [ ]:
class DataDesc:
    """數據描述符"""
    def __get__(self, obj, owner=None):
        return "DataDesc.__get__"

    def __set__(self, obj, value):
        print("DataDesc.__set__")

class NonDataDesc:
    """非數據描述符"""
    def __get__(self, obj, owner=None):
        return "NonDataDesc.__get__"

class Owner:
    data_desc = DataDesc()
    non_data_desc = NonDataDesc()

    def __init__(self):
        self.data_desc = "instance data_desc"
        self.non_data_desc = "instance non_data_desc"

def test_descriptor_types():
    obj = Owner()

    print("數據描述符：")
    print(f"- 通過實例訪問：{obj.data_desc}")
    print(f"- 通過類訪問：{Owner.data_desc}")

    print("\n非數據描述符：")
    print(f"- 通過實例訪問：{obj.non_data_desc}")
    print(f"- 通過類訪問：{Owner.non_data_desc}")

### 2.3 記憶體管理和性能考慮

In [ ]:
from weakref import WeakKeyDictionary

class EffectiveDescriptor:
    """使用 WeakKeyDictionary 的高效 descriptor 實現"""

    def __init__(self):
        # 使用 WeakKeyDictionary 避免記憶體洩漏
        self._values = WeakKeyDictionary()

    def __get__(self, obj, owner=None):
        if obj is None:
            return self
        return self._values.get(obj)

    def __set__(self, obj, value):
        self._values[obj] = value

class MemoryTest:
    attr = EffectiveDescriptor()

# 測試
def test_memory():
    import sys

    # 創建多個實例測試記憶體使用
    objects = [MemoryTest() for _ in range(1000)]
    for i, obj in enumerate(objects):
        obj.attr = f"value_{i}"

    # 檢查部分對象是否正確存儲值
    print(objects[0].attr)  # 應輸出 "value_0"
    print(objects[999].attr)  # 應輸出 "value_999"

    # 刪除部分對象，觀察記憶體使用
    del objects[::2]  # 刪除偶數索引的對象

## 3. 主要使用場景
### 3.1 屬性訪問控制


#### 3.1.1 基本屬性驗證
- 驗證邏輯集中在一處，數據驗證的一致性
- 可以在多個類中重用(重用性)
- 代碼更清晰易維護，增加使用的靈活性
- 自動執行驗證，不需要手動調用
- 錯誤處理的完整性

In [25]:
# 1. 基本的屬性驗證
class PositiveNumber:
    def __get__(self, obj, owner=None):
        return obj._value

    def __set__(self, obj, value):
        if value <= 0:
            raise ValueError("值必須是正數")
        obj._value = value

class Product:
    price = PositiveNumber()  # 使用 descriptor 控制 price 屬性

# 使用示例
product = Product()
product.price = 100   # 正確
try:
    product.price = -50  # 引發錯誤
except ValueError as e:
    print(f"錯誤：{e}")  # 輸出：錯誤：值必須是正數

錯誤：值必須是正數


這個例子展示了：
- 如何驗證屬性值
- 如何在設置值前進行檢查
- 如何拋出適當的錯誤
- descriptor 如何與類和實例互動

In [ ]:
class ValidatedPrice:
    def __init__(self, min_value=0, max_value=None, allow_none=False):
        self._min_value = min_value
        self._max_value = max_value
        self._allow_none = allow_none
        self._name = None

    def __set_name__(self, owner, name):
        self._name = name

    def __get__(self, obj, owner=None):
        if obj is None:
            return self
        return getattr(obj, f'_{self._name}', None)

    def __set__(self, obj, value):
        if value is None:
            if not self._allow_none:
                raise ValueError(f"{self._name} 不能為空")
            setattr(obj, f'_{self._name}', None)
            return

        # 類型檢查
        if not isinstance(value, (int, float)):
            raise TypeError(f"{self._name} 必須是數字")

        # 最小值檢查
        if value < self._min_value:
            raise ValueError(f"{self._name} 不能小於 {self._min_value}")

        # 最大值檢查
        if self._max_value is not None and value > self._max_value:
            raise ValueError(f"{self._name} 不能大於 {self._max_value}")

        # 檢查是否為整數
        if isinstance(value, float) and value.is_integer():
            value = int(value)  # 轉換成整數

        # 精度檢查（如果是浮點數）
        if isinstance(value, float):
            value = round(value, 2)  # 保留兩位小數

        setattr(obj, f'_{self._name}', value)

class Product:
    price = ValidatedPrice(min_value=0, max_value=1000000, allow_none=False)
    discount_price = ValidatedPrice(min_value=0, max_value=1000000, allow_none=True)

    def __init__(self, price, discount_price=None):
        self.price = price
        self.discount_price = discount_price

# 測試代碼
def test_product_price():
    print("\n1. 測試正常情況:")
    try:
        p = Product(100.567, 90)
        print(f"正常價格: {p.price}")  # 會自動四捨五入到 100.57
        print(f"折扣價格: {p.discount_price}")
    except Exception as e:
        print(f"錯誤: {e}")

    print("\n2. 測試空值:")
    try:
        p = Product(None)
    except ValueError as e:
        print(f"預期的錯誤: {e}")  # price 不能為空

    print("\n3. 測試類型錯誤:")
    try:
        p = Product("不是數字")
    except TypeError as e:
        print(f"預期的錯誤: {e}")

    print("\n4. 測試範圍錯誤:")
    try:
        p = Product(-50)
    except ValueError as e:
        print(f"預期的錯誤: {e}")

    try:
        p = Product(2000000)
    except ValueError as e:
        print(f"預期的錯誤: {e}")

    print("\n5. 測試折扣價格:")
    try:
        p = Product(100)
        p.discount_price = 150  # 折扣價格高於原價
        print("折扣價格設置成功（雖然高於原價）")
        p.discount_price = None  # 允許空值
        print("成功設置空折扣價格")
    except Exception as e:
        print(f"錯誤: {e}")

if __name__ == "__main__":
    test_product_price()

這個例子展示了：

1. 多種驗證條件：
- 類型檢查（必須是數字）
- 範圍檢查（最小值/最大值）
- 空值檢查（是否允許 None）
- 數值精度處理（四捨五入到小數點後兩位）
- 整數轉換（如果是整數值的浮點數）

2. 錯誤處理：
- TypeError：類型錯誤
- ValueError：值錯誤
- 具體的錯誤消息

3. 靈活性：
- 可配置的最小值/最大值
- 可配置是否允許空值
- 自動處理數值格式

4. 使用時的各種情況：
- 正常設置值
- 設置無效值
- 設置空值
- 設置不同類型的值
- 設置超出範圍的值

#### 3.1.2 只讀屬性
展示了如何控制屬性訪問（在這裡是禁止修改）

封裝完整：使用者無法繞過只讀限制

In [19]:
class ReadOnly:
    def __init__(self, value):
        self._value = value

    def __get__(self, obj, owner=None):
        return self._value

    def __set__(self, obj, value):
        raise AttributeError("此屬性是只讀的")

class Config:
    VERSION = ReadOnly("1.0.0")
    API_KEY = ReadOnly("secret_key")

#### 3.1.3 私有屬性

In [ ]:
class PrivateAttribute:
    def __init__(self):
        self._value = None
        self._name = None

    def __set_name__(self, owner, name):
        self._name = name

    def __get__(self, obj, owner=None):
        if obj is None:
            return self
        if not self._check_access(obj):
            raise PermissionError(f"無法訪問私有屬性 {self._name}")
        return self._value

    def __set__(self, obj, value):
        if not self._check_access(obj):
            raise PermissionError(f"無法修改私有屬性 {self._name}")
        self._value = value

    def _check_access(self, obj):
        import inspect
        frame = inspect.currentframe().f_back.f_back
        return frame.f_code.co_name in obj.__class__.__dict__

class User:
    _password = PrivateAttribute()

    def __init__(self, password):
        self._password = password

    def check_password(self, pwd):
        return self._password == pwd

#### 3.1.3 受限制的屬性

### 3.2 類型檢查和驗證
這個例子的特點：

1. 類型安全：確保屬性值符合預期類型
2. 實例隔離：使用 __dict__ 為每個實例分別存儲值
3. 清晰的錯誤信息：提供具體的類型錯誤提示
4. 可重用：同一個 descriptor 可用於不同屬性

這個例子比前面的 ReadOnly 更進階，因為它：

1. 使用了 __set_name__ 方法
2. 實現了實例值的獨立存儲
3. 提供了動態的類型檢查
4. 展示了更複雜的錯誤處理

這樣的類型檢查在以下場景特別有用：

- 數據模型定義
- API 參數驗證
- 配置項檢查
- 用戶輸入驗證

In [ ]:
class TypeChecked:
    def __init__(self, type_):
        self._type = type_
        self._name = None

    def __set_name__(self, owner, name):
        self._name = name

    def __get__(self, obj, owner=None):
        if obj is None:
            return self
        return obj.__dict__.get(self._name)

    def __set__(self, obj, value):
        if not isinstance(value, self._type):
            raise TypeError(f"{self._name} 必須是 {self._type.__name__} 類型")
        obj.__dict__[self._name] = value

class Person:
    name = TypeChecked(str)
    age = TypeChecked(int)

# 使用示例
person = Person()
person.name = "Alice"    # 正確
person.age = 25         # 正確
person.name = 123       # TypeError: name 必須是 str 類型
person.age = "25"       # TypeError: age 必須是 int 類型

### 3.3 訪問日誌和權限控制

#### 3.3.1 基本日誌記錄

In [ ]:
class LoggedAccess:
    def __init__(self):
        self._value = None
        self._name = None

    def __set_name__(self, owner, name):
        self._name = name

    def __get__(self, obj, owner=None):
        print(f"[訪問日誌] {obj.__class__.__name__}.{self._name} 被讀取")
        return self._value

    def __set__(self, obj, value):
        print(f"[訪問日誌] {obj.__class__.__name__}.{self._name} 被設置為 {value}")
        self._value = value

# 使用示例
class User:
    password = LoggedAccess()
    credit_card = LoggedAccess()

user = User()
user.password = "123456"      # 輸出: [訪問日誌] User.password 被設置為 123456
print(user.credit_card)       # 輸出: [訪問日誌] User.credit_card 被讀取

#### 3.3.2 權限控制

In [ ]:
class AccessControl:
    def __init__(self, permission_level=0):
        self._value = None
        self._permission_level = permission_level

    def __get__(self, obj, owner=None):
        if not self._check_permission(obj):
            raise PermissionError("權限不足，無法訪問此資料")
        return self._value

    def __set__(self, obj, value):
        if not self._check_permission(obj):
            raise PermissionError("權限不足，無法修改此資料")
        self._value = value

    def _check_permission(self, obj):
        return getattr(obj, 'user_level', 0) >= self._permission_level

# 使用示例
class Document:
    # 一般內容需要 level 1
    content = AccessControl(permission_level=1)
    # 機密內容需要 level 2
    secret_content = AccessControl(permission_level=2)

    def __init__(self, user_level):
        self.user_level = user_level

# 測試
doc = Document(user_level=1)
doc.content = "一般內容"     # 可以設置
try:
    doc.secret_content = "機密內容"  # 會拋出 PermissionError
except PermissionError as e:
    print(f"錯誤: {e}")

#### 3.3.3 結合日誌和權限

In [ ]:
class SecureLogging:
    def __init__(self, permission_level=0):
        self._value = None
        self._permission_level = permission_level
        self._name = None

    def __set_name__(self, owner, name):
        self._name = name

    def __get__(self, obj, owner=None):
        if self._check_permission(obj):
            print(f"[安全日誌] {obj.__class__.__name__}.{self._name} 被成功讀取")
            return self._value
        print(f"[安全日誌] 嘗試未經授權訪問 {self._name}")
        raise PermissionError("無訪問權限")

    def __set__(self, obj, value):
        if self._check_permission(obj):
            print(f"[安全日誌] {obj.__class__.__name__}.{self._name} 被設置為 {value}")
            self._value = value
        else:
            print(f"[安全日誌] 嘗試未經授權修改 {self._name}")
            raise PermissionError("無修改權限")

    def _check_permission(self, obj):
        return getattr(obj, 'user_level', 0) >= self._permission_level

# 使用示例
class SecureDocument:
    content = SecureLogging(permission_level=1)

    def __init__(self, user_level):
        self.user_level = user_level

# 測試
print("使用權限不足的用戶：")
doc1 = SecureDocument(user_level=0)
try:
    doc1.content = "測試內容"
except PermissionError as e:
    print(f"預期的錯誤: {e}")

print("\n使用有權限的用戶：")
doc2 = SecureDocument(user_level=1)
doc2.content = "機密內容"
print(doc2.content)

### 4.2 類型檢查和驗證
- 確保屬性值符合特定類型
- 可以設置值的範圍限制
- 提供清晰的錯誤信息

In [ ]:
class ValidatedProperty:
    def __init__(self, type_, min_value=None, max_value=None):
        self.type = type_
        self.min_value = min_value
        self.max_value = max_value
        self.name = None

    def __set_name__(self, owner, name):
        self.name = name

    def __get__(self, obj, owner=None):
        if obj is None:
            return self
        return obj.__dict__.get(self.name)

    def __set__(self, obj, value):
        if not isinstance(value, self.type):
            raise TypeError(f"{self.name} 必須是 {self.type.__name__} 類型")

        if self.min_value is not None and value < self.min_value:
            raise ValueError(f"{self.name} 不能小於 {self.min_value}")

        if self.max_value is not None and value > self.max_value:
            raise ValueError(f"{self.name} 不能大於 {self.max_value}")

        obj.__dict__[self.name] = value

class Person:
    age = ValidatedProperty(int, min_value=0, max_value=150)
    height = ValidatedProperty(float, min_value=0)
    name = ValidatedProperty(str)

In [ ]:
class TypeChecked:
    def __init__(self, type):
        self._type = type
        self._name = None

    def __set_name__(self, owner, name):
        # Python 3.6+ 特性，自動獲取屬性名
        self._name = name

    def __get__(self, obj, owner=None):
        if obj is None:
            return self
        return obj.__dict__.get(self._name)

    def __set__(self, obj, value):
        if not isinstance(value, self._type):
            raise TypeError(f"{self._name} 必須是 {self._type}")
        obj.__dict__[self._name] = value

class Person:
    age = TypeChecked(int)
    name = TypeChecked(str)

# 使用
person = Person()
person.age = 25  # 正常
person.name = "Alice"  # 正常
person.age = "不是數字"  # 引發 TypeError

### 4.3 計算屬性和緩存
#### 4.3.1 基本計算屬性

In [ ]:
class LazyProperty:
    def __init__(self, func):
        self.func = func

    def __get__(self, obj, owner=None):
        if obj is None:
            return self
        value = self.func(obj)
        setattr(obj, self.func.__name__, value)
        return value

class Circle:
    def __init__(self, radius):
        self.radius = radius

    @LazyProperty
    def area(self):
        print("計算面積...")  # 只會在第一次訪問時執行
        return 3.14 * self.radius ** 2

In [1]:
class LazyProperty:
    def __init__(self, func):
        self.func = func

    def __get__(self, obj, owner=None):
        if obj is None:
            return self
        # 計算值並緩存
        value = self.func(obj)
        setattr(obj, self.func.__name__, value)
        return value

class Circle:
    def __init__(self, radius):
        self.radius = radius

    @LazyProperty
    def area(self):
        print("計算面積...")  # 只會在第一次訪問時執行
        return 3.14 * self.radius ** 2

# 使用
c = Circle(5)
print(c.area)  # 第一次：計算並緩存
print(c.area)  # 第二次：直接返回緩存值

計算面積...
78.5
78.5


In [ ]:
class LazyProperty:
    def __init__(self, func):
        self.func = func

    def __get__(self, obj, owner=None):
        if obj is None:
            return self
        # 計算值並緩存
        value = self.func(obj)
        setattr(obj, self.func.__name__, value)
        return value

class Circle:
    def __init__(self, radius):
        self.radius = radius

    @LazyProperty
    def area(self):
        print("計算面積...")  # 只會在第一次訪問時執行
        return 3.14 * self.radius ** 2

# 使用
c = Circle(5)
print(c.area)  # 第一次：計算並緩存
print(c.area)  # 第二次：直接返回緩存值

#### 4.3.2 高級緩存屬性

In [ ]:
class CachedProperty:
    def __init__(self, func):
        self.func = func
        self.name = func.__name__

    def __get__(self, obj, owner=None):
        if obj is None:
            return self

        cache_name = f'_cached_{self.name}'
        if not hasattr(obj, cache_name):
            setattr(obj, cache_name, self.func(obj))
        return getattr(obj, cache_name)

class Circle:
    def __init__(self, radius):
        self.radius = radius

    @CachedProperty
    def area(self):
        print("計算圓的面積...")
        return 3.14 * self.radius ** 2

    @CachedProperty
    def circumference(self):
        print("計算圓的周長...")
        return 2 * 3.14 * self.radius

In [ ]:
class Field:
    def __init__(self, column_type):
        self.column_type = column_type
        self.name = None

    def __set_name__(self, owner, name):
        self.name = name

    def __get__(self, obj, owner=None):
        if obj is None:
            return self
        return obj._data.get(self.name)

    def __set__(self, obj, value):
        if not isinstance(value, self.column_type):
            raise TypeError(f"{self.name} 必須是 {self.column_type}")
        if not hasattr(obj, '_data'):
            obj._data = {}
        obj._data[self.name] = value

class Model:
    def __init__(self):
        self._data = {}

class User(Model):
    id = Field(int)
    name = Field(str)
    email = Field(str)

# 使用
user = User()
user.id = 1
user.name = "Alice"
user.email = "alice@example.com"

#### 4.3.3 高級緩存屬性 - 非數據描述符 vs 數據描述符：
- 實現複雜計算的緩存
- 避免重複計算
- 只在第一次訪問時執行計算

In [ ]:
class NonDataDescriptor:  # 只有 __get__
    def __get__(self, obj, owner=None):
        return "非數據描述符"

class DataDescriptor:  # 同時有 __get__ 和 __set__
    def __get__(self, obj, owner=None):
        return "數據描述符"

    def __set__(self, obj, value):
        print("設置值")

class Example:
    nd = NonDataDescriptor()
    d = DataDescriptor()

    def __init__(self):
        self.nd = "實例屬性"  # 實例屬性會覆蓋非數據描述符
        self.d = "實例屬性"   # 數據描述符優先級更高

### 4.4 訪問日誌和監控
- 記錄屬性的訪問和修改
- 可用於調試和監控
- 實現訪問控制邏輯

In [ ]:
class LoggedAccess:
    def __init__(self):
        self.name = None
        self._value = None

    def __set_name__(self, owner, name):
        self.name = name

    def __get__(self, obj, owner=None):
        if obj is None:
            return self
        print(f"訪問 {self.name}: 當前值 = {self._value}")
        return self._value

    def __set__(self, obj, value):
        print(f"設置 {self.name} = {value}")
        self._value = value

class User:
    name = LoggedAccess()
    email = LoggedAccess()

### 4.5 ORM 實現
- 數據庫欄位映射
- 類型驗證
- 提供數據持久化接口

In [ ]:
class Field:
    def __init__(self, column_type, primary_key=False):
        self.column_type = column_type
        self.primary_key = primary_key
        self.name = None

    def __set_name__(self, owner, name):
        self.name = name

    def __get__(self, obj, owner=None):
        if obj is None:
            return self
        return obj._data.get(self.name)

    def __set__(self, obj, value):
        if not isinstance(value, self.column_type):
            raise TypeError(f"{self.name} 必須是 {self.column_type}")
        if not hasattr(obj, '_data'):
            obj._data = {}
        obj._data[self.name] = value

class ModelMeta(type):
    def __new__(cls, name, bases, attrs):
        fields = {}
        for key, value in attrs.items():
            if isinstance(value, Field):
                fields[key] = value
        attrs['__fields__'] = fields
        return super().__new__(cls, name, bases, attrs)

class Model(metaclass=ModelMeta):
    def __init__(self):
        self._data = {}

    def save(self):
        fields = []
        values = []
        for name, field in self.__class__.__fields__.items():
            fields.append(name)
            values.append(getattr(self, name))
        print(f"INSERT INTO {self.__class__.__name__} ({','.join(fields)}) VALUES ({values})")

class UserModel(Model):
    id = Field(int, primary_key=True)
    name = Field(str)
    email = Field(str)

### 4.6 惰性加載
- 延遲加載大量數據
- 提高初始化效率
- 只在需要時才加載數據


In [ ]:
class LazyLoader:
    def __init__(self, loader_func):
        self.loader_func = loader_func
        self.name = None
        self._cached_value = None
        self._is_loaded = False

    def __set_name__(self, owner, name):
        self.name = name

    def __get__(self, obj, owner=None):
        if obj is None:
            return self

        if not self._is_loaded:
            print(f"惰性加載 {self.name}")
            self._cached_value = self.loader_func(obj)
            self._is_loaded = True
        return self._cached_value

class DataProcessor:
    def __init__(self, file_path):
        self.file_path = file_path

    @staticmethod
    def load_data(obj):
        print(f"從 {obj.file_path} 加載數據...")
        return ["大量數據..."]

    data = LazyLoader(load_data)

### 4.7 屬性代理和委託
- 將屬性訪問委託給其他對象
- 實現組合模式
- 簡化接口設計

In [ ]:
class DelegatedAttribute:
    def __init__(self, delegate_name, attr_name):
        self.delegate_name = delegate_name
        self.attr_name = attr_name

    def __get__(self, obj, owner=None):
        if obj is None:
            return self
        delegate = getattr(obj, self.delegate_name)
        return getattr(delegate, self.attr_name)

    def __set__(self, obj, value):
        delegate = getattr(obj, self.delegate_name)
        setattr(delegate, self.attr_name, value)

class InternalClass:
    def __init__(self):
        self.value = 0
        self.name = "Internal"

class MainClass:
    def __init__(self):
        self._internal = InternalClass()

    value = DelegatedAttribute('_internal', 'value')
    name = DelegatedAttribute('_internal', 'name')

In [ ]:
# 1. 屬性驗證和類型檢查
class ValidatedProperty:
    def __init__(self, type_, min_value=None, max_value=None):
        self.type = type_
        self.min_value = min_value
        self.max_value = max_value
        self.name = None

    def __set_name__(self, owner, name):
        self.name = name

    def __get__(self, obj, owner=None):
        if obj is None:
            return self
        return obj.__dict__.get(self.name)

    def __set__(self, obj, value):
        if not isinstance(value, self.type):
            raise TypeError(f"{self.name} 必須是 {self.type.__name__} 類型")

        if self.min_value is not None and value < self.min_value:
            raise ValueError(f"{self.name} 不能小於 {self.min_value}")

        if self.max_value is not None and value > self.max_value:
            raise ValueError(f"{self.name} 不能大於 {self.max_value}")

        obj.__dict__[self.name] = value

class Person:
    age = ValidatedProperty(int, min_value=0, max_value=150)
    height = ValidatedProperty(float, min_value=0)
    name = ValidatedProperty(str)

# 使用示例
person = Person()
person.age = 25
person.height = 175.5
person.name = "Alice"

# 2. 計算屬性和緩存
class CachedProperty:
    def __init__(self, func):
        self.func = func
        self.name = func.__name__

    def __get__(self, obj, owner=None):
        if obj is None:
            return self

        # 檢查是否已經計算過
        cache_name = f'_cached_{self.name}'
        if not hasattr(obj, cache_name):
            # 計算並緩存結果
            setattr(obj, cache_name, self.func(obj))
        return getattr(obj, cache_name)

class Circle:
    def __init__(self, radius):
        self.radius = radius

    @CachedProperty
    def area(self):
        print("計算圓的面積...")
        return 3.14 * self.radius ** 2

    @CachedProperty
    def circumference(self):
        print("計算圓的周長...")
        return 2 * 3.14 * self.radius

# 3. 訪問控制和日誌記錄
class LoggedAccess:
    def __init__(self):
        self.name = None
        self._value = None

    def __set_name__(self, owner, name):
        self.name = name

    def __get__(self, obj, owner=None):
        if obj is None:
            return self
        print(f"訪問 {self.name}: 當前值 = {self._value}")
        return self._value

    def __set__(self, obj, value):
        print(f"設置 {self.name} = {value}")
        self._value = value

class User:
    name = LoggedAccess()
    email = LoggedAccess()

# 4. ORM 實現示例
class Field:
    def __init__(self, column_type, primary_key=False):
        self.column_type = column_type
        self.primary_key = primary_key
        self.name = None

    def __set_name__(self, owner, name):
        self.name = name

    def __get__(self, obj, owner=None):
        if obj is None:
            return self
        return obj._data.get(self.name)

    def __set__(self, obj, value):
        if not hasattr(obj, '_data'):
            obj._data = {}
        if not isinstance(value, self.column_type):
            raise TypeError(f"{self.name} 必須是 {self.column_type}")
        obj._data[self.name] = value

class ModelMeta(type):
    def __new__(cls, name, bases, attrs):
        fields = {}
        for key, value in attrs.items():
            if isinstance(value, Field):
                fields[key] = value
        attrs['__fields__'] = fields
        return super().__new__(cls, name, bases, attrs)

class Model(metaclass=ModelMeta):
    def __init__(self):
        self._data = {}

    def save(self):
        # 模擬保存到數據庫
        fields = []
        values = []
        for name, field in self.__class__.__fields__.items():
            fields.append(name)
            values.append(getattr(self, name))
        print(f"INSERT INTO {self.__class__.__name__} ({','.join(fields)}) VALUES ({values})")

class UserModel(Model):
    id = Field(int, primary_key=True)
    name = Field(str)
    email = Field(str)

# 5. 惰性加載
class LazyLoader:
    def __init__(self, loader_func):
        self.loader_func = loader_func
        self.name = None
        self._cached_value = None
        self._is_loaded = False

    def __set_name__(self, owner, name):
        self.name = name

    def __get__(self, obj, owner=None):
        if obj is None:
            return self

        if not self._is_loaded:
            print(f"惰性加載 {self.name}")
            self._cached_value = self.loader_func(obj)
            self._is_loaded = True
        return self._cached_value

class DataProcessor:
    def __init__(self, file_path):
        self.file_path = file_path

    @staticmethod
    def load_data(obj):
        # 模擬從文件加載大量數據
        print(f"從 {obj.file_path} 加載數據...")
        return ["大量數據..."]

    data = LazyLoader(load_data)

# 6. 屬性代理和委託
class DelegatedAttribute:
    def __init__(self, delegate_name, attr_name):
        self.delegate_name = delegate_name
        self.attr_name = attr_name

    def __get__(self, obj, owner=None):
        if obj is None:
            return self
        delegate = getattr(obj, self.delegate_name)
        return getattr(delegate, self.attr_name)

    def __set__(self, obj, value):
        delegate = getattr(obj, self.delegate_name)
        setattr(delegate, self.attr_name, value)

class InternalClass:
    def __init__(self):
        self.value = 0
        self.name = "Internal"

class MainClass:
    def __init__(self):
        self._internal = InternalClass()

    # 代理內部對象的屬性
    value = DelegatedAttribute('_internal', 'value')
    name = DelegatedAttribute('_internal', 'name')

# 使用示例
def test_all_examples():
    # 1. 測試屬性驗證
    print("\n1. 測試屬性驗證:")
    person = Person()
    try:
        person.age = 25
        print(f"設置年齡成功: {person.age}")
        person.age = -1  # 會拋出異常
    except ValueError as e:
        print(f"驗證工作正常: {e}")

    # 2. 測試計算屬性
    print("\n2. 測試計算屬性:")
    circle = Circle(5)
    print(f"第一次訪問面積: {circle.area}")
    print(f"第二次訪問面積: {circle.area}")

    # 3. 測試訪問日誌
    print("\n3. 測試訪問日誌:")
    user = User()
    user.name = "Alice"
    _ = user.name

    # 4. 測試 ORM
    print("\n4. 測試 ORM:")
    user_model = UserModel()
    user_model.id = 1
    user_model.name = "Bob"
    user_model.email = "bob@example.com"
    user_model.save()

    # 5. 測試惰性加載
    print("\n5. 測試惰性加載:")
    processor = DataProcessor("big_data.txt")
    print("創建對象後，數據還未加載")
    print(f"訪問數據: {processor.data}")
    print(f"再次訪問: {processor.data}")

    # 6. 測試屬性代理
    print("\n6. 測試屬性代理:")
    main = MainClass()
    main.value = 42
    main.name = "New Name"
    print(f"代理的值: {main.value}")
    print(f"代理的名稱: {main.name}")

if __name__ == "__main__":
    test_all_examples()